Create First an empty histogram with 50 bins with range [-10,10]

In [ ]:
TH1D h1("h1","h1",50,-10,10)

Fill the histogram with 10000 Gaussian Random number with mean=1 and sigma=2

In [ ]:
for (int i = 0; i < 1000; ++i) { h1.Fill(gRandom->Gaus(1,2));}

We could also use the function h1.FillRandom("gaus"), but we need to set the right parameters of the Gaussian function before.

In [ ]:
h1.Draw(); gPad->Draw();

Before Fitting we need to create the fitting function and set its initial parameter values. 
For a Gaussian function in reality is not needed, because they can be computed automatically. 

In [ ]:
TF1 * f1 = nullptr; 

In [ ]:
f1 = new TF1("f1","gaus",-10,10);

In [ ]:
f1->SetParameters(100,0,1);  

We fit now the histogram using the Fit method in ROOT. By default the least-square method is used. 
For likelihood fits we need to use the option "L". 
The option "S" is used to create a TFitResult object that is returned to the user.
If we want to compute the error using MINOS, we use the "E" option
We want to change also the default Minimization engine. We will use Minuit2

In [ ]:
ROOT::Math::MinimizerOptions::SetDefaultMinimizer("Minuit2");

In [ ]:
TFitResultPtr res; 

In [ ]:
res = h1.Fit(f1, "S L E")

In [ ]:
gStyle->SetOptFit(1);

In [ ]:
gPad->Draw();

We can now print the result of the Fit

In [ ]:
res->Print();

We now get the corre;ation matrix of the fit from the TFitResult class

In [ ]:
TMatrixDSym corrMatrix;

In [ ]:
corrMatrix = res->GetCorrelationMatrix(); 

In [ ]:
corrMatrix.Print(); 

In [ ]:
std::cout  << "Gaussian sigma = " << f1->GetParameter("Sigma") << "  +/- " << f1->GetParError(f1->GetParNumber("Sigma")) << std::endl;

If we want to access the MINOS asymmetric error, we can get them from the FitResult object

In [ ]:
std::cout  << "Gaussian sigma = " << res->Parameter(2) << "  + " << res->UpperError(2) << "  " << res->LowerError(2) << std::endl;

We study now the Fit Bias for the sigma parameter

In [ ]:
TH1 * hs = nullptr;

In [ ]:
hs = new TH1D("hs","Sigma pull distribution",50,-4,4);

In [ ]:
hs->Reset(); // in case we run a second time
for (int iexp = 0; iexp < 1000; ++iexp) { 
    TH1D h0("h0","h0",50,-10,10);
    for (int i = 0; i < 1000; ++i){
        h0.Fill(gRandom->Gaus(0,2));
    }
    TF1 f1("f1","gaus",-10,10);
    f1.SetParameters(100,0,1);
    h0.Fit(&f1,"Q  ");  // use Q option to avoid too much printing
    hs->Fill( (f1.GetParameter(2)- 2)/f1.GetParError(2));  // to get sigma and its error
    //std::cout << f1.GetParameter(2) << " " << f1.GetParError(2) << std::endl;
}

In [ ]:
hs->Fit("gaus"); gPad->Draw();